# target이 30억 이상인 데이터에 대해서 따로 모델 구축

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import gc

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import joblib

# 맥에서 글씨 깨짐 방지
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# data load

In [2]:
df = pd.read_csv('../data/preprocessed/34_apart_price_mean.csv')

/tmp/ipykernel_637308/313617256.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/32_selling_point.csv')


# data 전처리

In [3]:
# 일부 feature 제거 - 기준은 내마음대로

df.drop(['해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호',
         'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
         'k-사용검사일-사용승인일', 'k-관리비부과면적', 'k-전용면적별세대현황(60이하)', 'k-전용면적별세대현황(60~85이하)',
         'k-85~135이하', 'k-135초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태',
         '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일'], axis=1, inplace=True)

In [4]:
# 추가로 제거가 필요한 feature 제거
df.drop(['시군구', '번지', '본번', '부번', '아파트명', '도로명', 'k-관리방식', 'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적',
         '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '시군구 번지', '좌표X,좌표Y', 'index', '역사_ID', '역사명', '위도', '경도',
         '가장 가까운 버스 정류장 index', '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호', '가장 가까운 버스 정류소명',
         '가장 가까운 버스 정류소 타입', '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표', '동', 
         '구', '호선'], axis=1, inplace=True)

In [5]:
# 계약월 변수 생성

df['계약월'] = df['계약년월'].astype(str).str[4:].astype(int)
df.drop(['계약년월', '계약일'], axis=1, inplace=True)
df['계약월_sin'] = np.sin(2 * np.pi * df['계약월'] / 12)
df.drop(['계약월'], axis=1, inplace=True)
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,가장 가까운 다리 index,가장 가까운 다리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군,매매가격 지수,계약월_sin
0,79.97,3,1987,127.05721,37.476763,124000.0,0,1127.738351,2.0,61.89584,...,강남구,기타,5581.622466,17.0,청담대교,0.0,0,8.0,67.392545,-2.449294e-16
1,79.97,4,1987,127.05721,37.476763,123500.0,0,1127.738351,2.0,61.89584,...,강남구,기타,5581.622466,17.0,청담대교,0.0,0,8.0,67.392545,-2.449294e-16
2,54.98,5,1987,127.05721,37.476763,91500.0,0,1127.738351,2.0,61.89584,...,강남구,기타,5581.622466,17.0,청담대교,0.0,0,8.0,67.392545,-2.449294e-16
3,79.97,4,1987,127.05721,37.476763,130000.0,0,1127.738351,2.0,61.89584,...,강남구,기타,5581.622466,17.0,청담대교,0.0,0,8.0,69.049924,5.000000e-01
4,79.97,2,1987,127.05721,37.476763,117000.0,0,1127.738351,2.0,61.89584,...,강남구,기타,5581.622466,17.0,청담대교,0.0,0,8.0,69.049924,5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,84.65,13,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,기타,기타,5918.624352,23.0,구리암사대교,0.0,0,1.0,91.623694,-5.000000e-01
1128090,84.62,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,기타,기타,5918.624352,23.0,구리암사대교,0.0,0,1.0,91.623694,-5.000000e-01
1128091,101.65,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,기타,기타,5918.624352,23.0,구리암사대교,0.0,0,1.0,91.066919,-8.660254e-01
1128092,84.94,18,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,기타,기타,5918.624352,23.0,구리암사대교,0.0,0,1.0,90.861265,-1.000000e+00


# scaling

In [6]:
def scailing(col, scaler_type):
    if scaler_type == 'min_max': scaler = MinMaxScaler()
    else : scaler = StandardScaler()

    return scaler.fit_transform(df[[col]])

cols = ['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '가장 가까운 다리와의 거리', '매매가격 지수', 
       '아파트 평균 가격', '계약월_sin']

for col in cols:
    print(col)
    scaler_type = 'min_max'
    if col == '계약월_sin': scaler_type = 'standard'

    df[[col]] = scailing(col, scaler_type)

전용면적
층
건축년도
좌표X
좌표Y
가장 가까운 거리
인근 지하철 역 개수
가장 가까운 버스 정류장 거리
인근 버스 정류장 개수
GDP
한국은행 기준금리
기대 인플레이션
지가지수
아파트 인허가
미분양
거래량
구별 지가지수
공시지가 평균
매수우위지수
건물나이
구매력지수
거래활발지수
매매가격 지수 증감률
매매 대비 전세가격 비율
가장 가까운 다리와의 거리
매매가격 지수
계약월_sin


# category

In [7]:
#df['30년이상50년이하'] = df['30년이상50년이하'].astype('category')
df['아파트 카테고리'] = df['아파트 카테고리'].astype('category')
df['지하철 카테고리'] = df['지하철 카테고리'].astype('category')
df['구 카테고리'] = df['구 카테고리'].astype('category')
df['건설사 카테고리'] = df['건설사 카테고리'].astype('category')
df['인근 다리 개수 1개이상 3개 이하'] = df['인근 다리 개수 1개이상 3개 이하'].astype('category')
df['학군'] = df['학군'].astype('category')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128094 entries, 0 to 1128093
Data columns (total 41 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   전용면적                 1128094 non-null  float64 
 1   층                    1128094 non-null  float64 
 2   건축년도                 1128094 non-null  float64 
 3   좌표X                  1128094 non-null  float64 
 4   좌표Y                  1128094 non-null  float64 
 5   target               1118822 non-null  float64 
 6   is_test              1128094 non-null  int64   
 7   가장 가까운 거리            1128094 non-null  float64 
 8   인근 지하철 역 개수          1128094 non-null  float64 
 9   가장 가까운 버스 정류장 거리     1128094 non-null  float64 
 10  인근 버스 정류장 개수         1128094 non-null  float64 
 11  계약년                  1128094 non-null  int64   
 12  GDP                  1128094 non-null  float64 
 13  한국은행 기준금리            1128094 non-null  float64 
 14  기대 인플레이션             1128094 non-n

# train, test dataset

## feature drop

In [40]:
train_df = df[df['is_test']==0]
test_df = df[df['is_test']==1]

In [41]:
train_df.drop(['is_test'], axis=1, inplace=True)
test_df.drop(['is_test'], axis=1, inplace=True)

/tmp/ipykernel_637308/1571330888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(['is_test'], axis=1, inplace=True)
/tmp/ipykernel_637308/1571330888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['is_test'], axis=1, inplace=True)


In [42]:
print(train_df.shape)
print(test_df.shape)

(1118822, 40)
(9272, 40)


In [43]:
train_df_1 = train_df
train_df_2 = train_df[train_df['target'] >= 300000]

### target이 30억 이상인 데이터 분할

In [44]:
train_df_1_dropped = train_df_1.drop(['지가지수', '아파트 인허가', '미분양', '건설사 랭킹',
                      '구매력지수', '거래활발지수', '건물나이', '매매가격 지수 증감률',
                      '매매 대비 전세가격 비율', '구 카테고리',
                      '건설사 카테고리', '가장 가까운 다리 index', '가장 가까운 다리', '인근 한강다리 개수',
                      '인근 다리 개수 1개이상 3개 이하'], axis=1)

train_df_2_dropped = train_df_2.drop(['지가지수', '아파트 인허가', '미분양', '건설사 랭킹',
                      '구매력지수', '거래활발지수', '건물나이', '매매가격 지수 증감률',
                      '매매 대비 전세가격 비율', '구 카테고리',
                      '건설사 카테고리', '가장 가까운 다리 index', '가장 가까운 다리', '인근 한강다리 개수',
                      '인근 다리 개수 1개이상 3개 이하'], axis=1)

test_df_dropped = test_df.drop(['target', '지가지수', '아파트 인허가', '미분양', '건설사 랭킹',
                      '구매력지수', '거래활발지수', '건물나이', '매매가격 지수 증감률',
                      '매매 대비 전세가격 비율', '구 카테고리',
                      '건설사 카테고리', '가장 가까운 다리 index', '가장 가까운 다리', '인근 한강다리 개수',
                      '인근 다리 개수 1개이상 3개 이하'], axis=1)

In [46]:
train_df_1_dropped[['target']]

,target
0,124000.0
1,123500.0
2,91500.0
3,130000.0
4,117000.0
...,...
1118817,20000.0
1118818,20000.0
1118819,28000.0
1118820,29000.0


In [47]:
train_df_2_dropped[['target']]

,target
294,300000.0
647,300000.0
1308,395000.0
1309,390000.0
1310,443000.0
...,...
1078852,301000.0
1078853,310500.0
1105621,400000.0
1105660,450000.0


In [48]:
X_1 = train_df_1_dropped.drop(['target'], axis=1)
y_1 = train_df_1_dropped[['target']]

X_2 = train_df_2_dropped.drop(['target'], axis=1)
y_2 = train_df_2_dropped[['target']]

X_1.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리', '인근 지하철 역 개수',
       '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP', '한국은행 기준금리',
       '기대 인플레이션', '거래량', '구별 지가지수', '공시지가 평균', '매수우위지수', '30년이상50년이하',
       '아파트 카테고리', '지하철 카테고리', '가장 가까운 다리와의 거리', '학군', '매매가격 지수', '계약월_sin'],
      dtype='object')

# model

### 30억 이하

In [49]:
train_X_1, valid_X_1, train_y_1, valid_y_1 = train_test_split(X_1, y_1, train_size=0.95, test_size=0.05, random_state=74)

In [50]:
params = {
    'n_estimators': 100000,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'metric_freq': 20,
    'device': 'gpu',
    'verbosity': 0
}

model_1 = lgb.LGBMRegressor(n_estimators=100000,
                          metric='rmse', data_sample_strategy='goss'
                          )

model_1.fit(
    train_X_1, train_y_1,
    eval_set = [(train_X_1, train_y_1), (valid_X_1, valid_y_1)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=60),
               lgb.log_evaluation(period=10, show_stdv=True)]
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014521 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2972
[LightGBM] [Info] Number of data points in the train set: 1062880, number of used features: 24
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 57982.908459
Training until validation scores don't improve for 60 rounds
[10]	training's rmse: 26249.5	valid_1's rmse: 26853.1
[20]	training's rmse: 19122.6	valid_1's rmse: 19489.3
[30]	training's rmse: 16084.7	valid_1's rmse: 16342.7
[40]	training's rmse: 14533.8	valid_1's rmse: 14755.5
[50]	training's rmse: 13569	valid_1's rmse: 13809.2
[60]	training's rmse: 12970	valid_1's rmse: 13212.6
[70]	training's rm

LGBMRegressor(data_sample_strategy='goss', metric='rmse', n_estimators=100000)

### 30억 이상

In [51]:
train_X_2, valid_X_2, train_y_2, valid_y_2 = train_test_split(X_2, y_2, train_size=0.95, test_size=0.05, random_state=74)

In [53]:
params = {
    'n_estimators': 100000,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'metric_freq': 20,
    'device': 'gpu',
    'verbosity': 0
}

model_2 = lgb.LGBMRegressor(n_estimators=100000,
                          metric='rmse', data_sample_strategy='goss'
                          )

model_2.fit(
    train_X_2, train_y_2,
    eval_set = [(train_X_2, train_y_2), (valid_X_2, valid_y_2)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=90),
               lgb.log_evaluation(period=10, show_stdv=True)]
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000424 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2441
[LightGBM] [Info] Number of data points in the train set: 4703, number of used features: 24
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 389272.251967
Training until validation scores don't improve for 90 rounds
[10]	training's rmse: 62148.1	valid_1's rmse: 72386.9
[20]	training's rmse: 46172.1	valid_1's rmse: 56977
[30]	training's rmse: 38664.5	valid_1's rmse: 50123.8
[40]	training's rmse: 34790.4	valid_1's rmse: 46042.8
[50]	training's rmse: 32342.1	valid_1's rmse: 44317
[60]	training's rmse: 30537.9	valid_1's rmse: 43445.1
[70]	training's rmse: 29109.8	valid_1's rmse: 42666.4
[80]	training's rmse: 27809.1	

LGBMRegressor(data_sample_strategy='goss', metric='rmse', n_estimators=100000)

In [54]:
valid_X_2.shape

(248, 24)

In [55]:
preds_1 = model_1.predict(test_df_dropped)

In [56]:
preds_2 = model_2.predict(test_df_dropped)

In [57]:
preds_1

array([187367.39534053, 301817.77561001, 345225.44560216, ...,
        80885.08856464,  74841.30189952,  72799.11421885])

In [58]:
preds_2

array([276729.14655557, 363433.80153425, 354964.35626491, ...,
       246924.8924215 , 234714.02040978, 237670.87017157])

In [60]:
preds_1 = np.round(preds_1).astype(int)
preds_2 = np.round(preds_2).astype(int)

In [61]:
joblib.dump(model_1, './v_32_5_1.pkl')
joblib.dump(model_2, './v_32_5_2.pkl')

['./v_32_5_2.pkl']

In [67]:
preds_1_df = pd.DataFrame(preds_1, columns=['target'])
preds_2_df = pd.DataFrame(preds_2, columns=['target'])

preds_1_df.to_csv('../preds/v_32_5_1_preds.csv', index=False)
preds_2_df.to_csv('../preds/v_32_5_2_preds.csv', index=False)